# Decision tree exercise

The goal is to construct a small yet useful decision tree that predicts whether a website is a
phishing site or not.
Your result must contain all of the following:
1. An image of the final decision tree.
2. Written instructions for an internet analyst to make the decision of whether the website
is likely to be a phishing site or not. The instructions must match one-to-one with your
decision tree.
3. The accuracy estimate (percentage of correct classifications) of your decision tree.
4. A copy of the Python code used.

In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
import graphviz
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

from IPython.display import IFrame
import os
os.environ["PATH"] += os.pathsep + 'c:/Program Files (x86)/Graphviz2.38/bin/'


# Decision tree for Phising data

# read data
df = pd.read_csv('phishing.csv', sep=';')



df.head(10)


,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWindow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
5,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,-1,1,-1,-1,1
6,1,0,-1,1,1,-1,-1,-1,1,1,...,1,1,1,-1,-1,-1,1,0,-1,-1
7,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,0,-1,1,0,1,-1
8,1,0,-1,1,1,-1,1,1,-1,1,...,1,1,1,-1,1,1,1,0,1,1
9,1,1,-1,1,1,-1,-1,1,-1,1,...,1,1,1,-1,0,-1,1,0,1,-1


In [2]:
# save column headings into a list
colnames = df.columns.get_values()
colnames

array(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State',
       'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token',
       'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH',
       'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover',
       'RightClick', 'popUpWindow', 'Iframe', 'age_of_domain',
       'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index',
       'Links_pointing_to_page', 'Statistical_report', 'Result'],
      dtype=object)

In [3]:
# extract explanatory variables into a data frame
X = df.loc[:, 'having_IP_Address':'Statistical_report']

# extract response variable (class variable) into a series
Y = df.loc[:, 'Result']

In [4]:
# decision tree classification
classifier = tree.DecisionTreeClassifier(max_depth=3)
classifier.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [5]:
# visualize
dot_data = tree.export_graphviz(classifier, out_file=None, feature_names=colnames[:30], class_names=['no','yes'])
graph = graphviz.Source(dot_data) 
graph.render("phishing4")

IFrame("phishing4.pdf", width=900, height=700)


In [6]:
# predict
Y_pred = classifier.predict(X)

# output confusion matrix
cm = confusion_matrix(Y, Y_pred)
print("Confusion matrix:\n",cm)

accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])
print("Accuracy calculated from the training set = %.3f" % (accuracy))

print(classification_report(Y, Y_pred, target_names=['no', 'yes']))

Confusion matrix:
 [[4425  473]
 [ 545 5612]]
Accuracy calculated from the training set = 0.908
              precision    recall  f1-score   support

          no       0.89      0.90      0.90      4898
         yes       0.92      0.91      0.92      6157

   micro avg       0.91      0.91      0.91     11055
   macro avg       0.91      0.91      0.91     11055
weighted avg       0.91      0.91      0.91     11055



In [7]:
# cross-validate
# k = number of folds
k = 10
scores = cross_val_score(estimator=classifier,
                        X=X,
                        y=Y,
                        scoring="accuracy",
                        cv=k)
print("Accuracies from %d individual folds:" % k)
print(scores)
print("Accuracy calculated using %d-fold cross validation = %.3f" % (k, scores.mean()))

Accuracies from 10 individual folds:
[0.88607595 0.88698011 0.90506329 0.89783002 0.90777577 0.92224231
 0.90506329 0.90135747 0.91394928 0.91485507]
Accuracy calculated using 10-fold cross validation = 0.904


In [14]:
# Experiment with min_samples_leaf to find the best model.

for leaf in range(201,-1,-20):
    print("min_samples_leaf: %d" % leaf)
    classifier = tree.DecisionTreeClassifier(min_samples_leaf=leaf)
    classifier.fit(X,Y)
    Y_pred = classifier.predict(X)
    cm = confusion_matrix(Y, Y_pred)
    accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])
    print("  Accuracy calculated from the training set = %.3f" % (accuracy))
    
    scores = cross_val_score(estimator=classifier,
                        X=X,
                        y=Y,
                        scoring="accuracy",
                        cv=k)
    print("  Accuracy calculated using %d-fold cross validation = %.3f" % (k, scores.mean()))
    
    

min_samples_leaf: 201
  Accuracy calculated from the training set = 0.923
  Accuracy calculated using 10-fold cross validation = 0.918
min_samples_leaf: 181
  Accuracy calculated from the training set = 0.922
  Accuracy calculated using 10-fold cross validation = 0.918
min_samples_leaf: 161
  Accuracy calculated from the training set = 0.925
  Accuracy calculated using 10-fold cross validation = 0.917
min_samples_leaf: 141
  Accuracy calculated from the training set = 0.925
  Accuracy calculated using 10-fold cross validation = 0.918
min_samples_leaf: 121
  Accuracy calculated from the training set = 0.926
  Accuracy calculated using 10-fold cross validation = 0.920
min_samples_leaf: 101
  Accuracy calculated from the training set = 0.929
  Accuracy calculated using 10-fold cross validation = 0.923
min_samples_leaf: 81
  Accuracy calculated from the training set = 0.930
  Accuracy calculated using 10-fold cross validation = 0.924
min_samples_leaf: 61
  Accuracy calculated from the trai